In [34]:
# 텐서플로와 다른 라이브러리 임포트
!pip install tensorflow-gpu==2.0.0
import tensorflow as tf

import numpy as np
import os
import time
import re

In [35]:
# 건강정보 데이터셋 다운로드
path_to_file = tf.keras.utils.get_file('1.txt', 'https://www.dropbox.com/sh/9ylm5nwnaxxgyjj/AAC3OsCJMGnpLl-n1A80q-Cja?dl=0&preview=1.txt')

In [36]:
# 디코딩
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('텍스트의 길이: {}자'.format(len(text)))
print(text[:250])

텍스트의 길이: 455858자
<!DOCTYPE html><html class="maestro" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><script nonce="84aTHgKaF//+TbMWiX80">
window._goch_ = {};
window.addEventListener('click', function(event) {
    'use strict';
    for (var elm = event.targ


In [37]:
def cleanText(readData):

    text = re.sub("[①②③④×：■∼▹「」□1234567890-=+,#/\?:^$@*\_.※~&%·ㆍ!』\\|\(\)\[\]\<\>\…》abcdefghijklmnopqrstuvwxyzABCDEFGHIZJLMNOPQRSTUVWXYZ\x01◦-]", '', readData)
    text = re.sub('\n+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

data = cleanText(text)
print(data)
f = open("new_text.txt", "w") # 파일 저장
f.write(data)
f.close()

 "" "" "" "K" {} '' { ' ' { { } } } "" "" "" "" "" " " "" "" 전처리 " " "" "" "" "" "K" { "" } "" "K" {" " {{ }{ { }}{ { }}{ { }}}{{}{ { {} }}{ }{ }}} "K" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" { } { } ' '' '' '' { } '' { } { } { } { } { } { } { '' } "" "" "K" { "" } "" "K"{" " " " {{} {}{ }} { {} } {{ }}"" "" { { {}""} { {} {}}{""}{ }{ {}{}}{ }{ }{"" "" "" "" "" }} { { " '""'" "" { " '""' '""' '" "'"}{ " '""' '""' '""'"}}} { " """ " "" "{""""}{"""""""" " '""'"} } {" "{""{""}} " '' '' '" "'" { "" " '' '' '" "'"}} {" "{""} " '' '' '" "'" { "" " '' '' '" "'"}} {" "{""{""}{""}}} { " " {}} {""{""""{""}""""{""}""{""}""} " '' " " '' " " '' """"" " "" " " '' "} {} { {{}} { }{ {}"" "" { }" "{{}}}{ {} { {}} }}"""""""""" { { }} { "" } { ""{ { ""{ " "} }}} { { " "} { }{ { } " "" "}{""""}}{}{} { { { {}{{""}}}{ """"{}{{""}}}}{}} { "" { """"{}{{""}}""}{}} { {} { }{ " "}{ {}}{ {{} { """"}} }} {{ {}{}{{" '' ''"""""{}}{}{}}}""} { {} {}{ {}}{ { """"}}{ {}}}{ {} {}{ {}}{ " ' "}} {{{ " ' " {}"

In [38]:
# 고유 문자수
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))

# 고유 문자에서 인덱스로 매핑 생성
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

고유 문자수 96개


In [39]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

print ('{} ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '%' :   5,
  '&' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  ...
}
'<!DOCTYPE htm' ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > [28  2 36 47 35 52 57 48 37  1 71 83 76]


In [40]:
# 단일 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 훈련 샘플/타깃 만들기
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

<
!
D
O
C


In [41]:
# 시퀀스 변환
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'<!DOCTYPE html><html class="maestro" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><script'
' nonce="84aTHgKaF//+TbMWiX80">\nwindow._goch_ = {};\nwindow.addEventListener(\'click\', function(event) {'
"\n    'use strict';\n    for (var elm = event.target; elm; elm = elm.parentElement) {\n        if (elm.i"
'd &&\n            window._goch_.hasOwnProperty(elm.id) &&\n            window._goch_[elm.id].call(elm, '
'event) === false) {\n            event.preventDefault();\n        }\n    }\n}, true);\nwindow._csp_externa'


In [42]:
# 텍스트 복사 및 이동
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [43]:
# 첫번째 훈련 샘플
for input_example, target_example in  dataset.take(1):
  print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

입력 데이터:  '<!DOCTYPE html><html class="maestro" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><scrip'
타깃 데이터:  '!DOCTYPE html><html class="maestro" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><script'


In [44]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  예상 출력: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0단계
  입력: 28 ('<')
  예상 출력: 2 ('!')
   1단계
  입력: 2 ('!')
  예상 출력: 36 ('D')
   2단계
  입력: 36 ('D')
  예상 출력: 47 ('O')
   3단계
  입력: 47 ('O')
  예상 출력: 35 ('C')
   4단계
  입력: 35 ('C')
  예상 출력: 52 ('T')


In [45]:
# 훈련 배치 크기
BATCH_SIZE = 64

# 데이터셋을 섞을 버퍼 크기
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [46]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(vocab)

# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

In [47]:
# 모델 설계
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model
  
  model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

  for input_example_batch, target_example_batch in dataset.take(1):
      example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

model.summary()

In [48]:
model.summary()

NameError: ignored